In [ ]:
# setup
# !pip3 install praw
import praw
import pandas as pd
import os
secret = os.environ.get("SECRET") # how to get: https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example#first-steps
client_id = os.environ.get("CLIENT_ID")
user_agent = os.environ.get("USER_AGENT")

/Users/preethi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import praw

def get_last_n_submissions(subreddit_name, n, client_id, client_secret, user_agent, flair=None):
    """
    Fetches the last `n` submissions from a given subreddit, optionally filtering by flair.

    Parameters:
    - subreddit_name (str): The name of the subreddit.
    - n (int): The number of submissions to fetch.
    - client_id (str): Reddit API client ID.
    - client_secret (str): Reddit API client secret.
    - user_agent (str): Reddit API user agent.
    - flair (str, optional): The flair to filter by (case-sensitive).

    Returns:
    - List of dictionaries with 'title', 'selftext', and 'flair'.
    """
    # Initialize Reddit API client
    reddit = praw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent
    )

    subreddit = reddit.subreddit(subreddit_name)
    submissions = []

    # Get last n submissions sorted by new
    count = 0
    for submission in subreddit.new(limit=100):  # Fetch more to account for filtering
        if flair and submission.link_flair_text not in flair:
            print('skipped: {}'.format(submission.link_flair_text))
            continue  # Skip if flair doesn't match

        submissions.append({
        "title": submission.title,
        "selftext": submission.selftext,
        "flair": submission.link_flair_text,
        "posted_time": submission.created_utc,  # Unix timestamp
        "upvotes": submission.score,
        "num_comments": submission.num_comments,
        "submission_id": submission.id,
        "upvote_ratio": submission.upvote_ratio,
        "over_18": submission.over_18
      })

        count += 1
        if count >= n:  # Stop when we reach `n` valid submissions
            break

    return submissions


In [ ]:
# my filters
n = 10
subreddits = ['casualconversation', 'seriousconversation']
flairs = {
    subreddits[0]:
    [
    ":star: Neat",
    # ":topic: Weekly Topic",      # From URL encoding
    ":gaming: Gaming",           # From URL encoding
    ":chat: Just Chatting",      # From URL encoding
    ":note: Music",              # From URL encoding
    ":film: Movies & Shows",     # From URL encoding
    ":food: Food & Drinks",      # From URL encoding
    ":paw: Pets & Animals",      # From URL encoding
    ":mark: Books & Reading",    # From URL encoding
    ":story: Life Stories",      # From URL encoding
    ":question: Questions",      # From URL encoding
    ":thinking: Thoughts & Ideas",  # From URL encoding
    ":ididit: Made did it"       # From URL encoding
]
,
    subreddits[1]: [
    ":drugs: Drugs & Alcohol",
    ":question: Question",
    ":mental_health: Mental Health",
    ":current_event: Current Event",
    ":general: Serious Discussion",
    # ":mod_post: Mod Post",
    ":religion: Religion",
    ":gender: Gender & Sexuality",
    ":opinion: Opinion",
    ":advice: Situational Advice",
    ":politics: Politics"
]
}

In [5]:
# run - main
for subreddit in subreddits:
  data = []
  submissions = get_last_n_submissions(subreddit, n, client_id, secret, user_agent, flairs[subreddit])
  print(submissions)
  for s in submissions:
    data.append({
            "title": s["title"],
            "text": s["selftext"],
            "flair": s["flair"],
            "posted_time": s["posted_time"],  # Unix timestamp
            "upvotes": s["upvotes"],
            "num_comments": s["num_comments"],
            "submission_id": s["submission_id"],
            "upvote_ratio": s["upvote_ratio"],
            "over_18": s["over_18"]
        })
  df = pd.DataFrame(data)
  df.to_csv('{}.csv'.format(subreddit))

skipped: None
skipped: None
skipped: None
skipped: None
skipped: None
skipped: None
skipped: None
skipped: None
skipped: None
skipped: None
skipped: :party: Celebration
skipped: None
skipped: None
[{'title': 'What’s a completely harmless thing that irrationally annoys you?', 'selftext': 'I’m talking small, stupid stuff that shouldn’t make you mad… but absolutely does.\n\nFor me? When someone starts telling me a story and then goes “oh nevermind.”\n\nNo. You don’t get to do that. I need closure now.\n\nWhat’s yours?', 'flair': ':chat: Just Chatting', 'posted_time': 1742762846.0, 'upvotes': 17, 'num_comments': 18, 'submission_id': '1ji9bva', 'upvote_ratio': 1.0, 'over_18': False}, {'title': 'What are your thoughts on my ideal city?', 'selftext': 'If I made a city from scratch, had all creative control with an unlimited budget:\nI’d prioritize walkability and public transit, and try to incorporate more nature into cities.  I’d also try to plant Miyawaki (aka tiny forests) forests to boost